In [1]:
%pip install -q git+https://github.com/huggingface/transformers.git
%pip install -q tensorflow==2.8
%pip install transformers

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 90.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 497.6/497.6 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 102.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.5/462.5 kB 39.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 67.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 96.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import tensorflow as tf
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer


In [4]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# add the EOS token as PAD token to avoid warnings
model = TFGPT2LMHeadModel.from_pretrained("gpt2", pad_token_id=tokenizer.eos_token_id)

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at gpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [5]:
import pandas as pd

In [6]:
df = pd.read_csv('/content/drive/MyDrive/ANLP/Data/eventwise_csv/FIVE.csv')

In [7]:
df.head

<bound method NDFrame.head of    Unnamed: 0  Match_id           Team  Over_num  \
0         629   3000439  KXIP 1st Inns       4.1   
1        4099   9886489    MI 1st Inns       4.5   
2        8113   7815415   KKR 1st Inns       0.5   

                                          Commentary           batsman score  \
0  Narine to Vohra, 5 wides, an inswinger from Na...   Narine to Vohra  FIVE   
1  Siraj to Lewis, 5 wides, poor from Siraj and n...    Siraj to Lewis  FIVE   
2  Chahar to Narine, 5 runs, direct hit would hav...  Chahar to Narine  FIVE   

                                      Commentary_new Team_new      P2      P1  \
0  {{BOWLER}} to {{BATSMAN}}, 5 wides, an inswing...     KXIP  Narine   Vohra   
1  {{BOWLER}} to {{BATSMAN}}, 5 wides, poor from ...       MI   Siraj   Lewis   
2  {{BOWLER}} to {{BATSMAN}}, 5 runs, direct hit ...      KKR  Chahar  Narine   

    P1_FN  P1_LN   P2_FN  P2_LN  P1_FN_count  P1_LN_count  P2_FN_count  \
0   Vohra    NaN  Narine    NaN          

## **Ignored due to insufficient data**

In [8]:
import os
os.makedirs("./input", exist_ok = False)


In [9]:
os.makedirs("./output", exist_ok = False)

In [10]:
len(df)

3

In [11]:
for i in range(len(df)):
    commentary = df.Commentary_new[i]               # This is the text of row 'i'.
    f = open("./input/" + str(i) + ".txt", "w")   # This writes a new text file.
    f.write(commentary)                             
    f.close()  

In [12]:
mypath = './input/'
import os
from os import listdir
from os.path import isfile, join
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
print(onlyfiles)
corpus = ''
for i in range(len(onlyfiles)):
  if onlyfiles[i][-3:] == 'txt':
    try:
      with open(mypath+ onlyfiles[i], 'r') as file:
        data = file.read()
      corpus = corpus + '\n' + data
    except:
      pass

['1.txt', '0.txt', '2.txt']


In [13]:
print(corpus)


{{BOWLER}} to {{BATSMAN}}, 5 wides, poor from {{BOWLER}} and nothing much de Kock can do about the bad bounce. Tries the variation and it slips outside off, could say that the wide call was marginal but the uneven bounce means that de Kock couldn't get down in time. Races away to the fence
{{BOWLER}} to {{BATSMAN}}, 5 wides, an inswinger from {{BOWLER}}. But this one goes a long way in the air, falls away down the leg-side and beats Uthappa completely. There's no stopping those runs
{{BOWLER}} to {{BATSMAN}}, 5 runs, direct hit would have taken out {{BATSMAN}}! Rayudu the man at mid-on but he can't get it on target and what's worse, there is nobody backing up. Races away to the extra cover fence. Bonus runs for {{TEAM}}. Was worked softly with a straightish bat but it was a risky run and {{BATSMAN}} is lucky to survive


In [14]:

import logging
import os
import pickle
import random
import torch
import torch.nn as nn
import transformers
from torch.utils.data import DataLoader, Dataset, RandomSampler, SequentialSampler
from transformers import (
    GPT2Config,
    GPT2LMHeadModel,
    GPT2PreTrainedModel,
    GPT2Tokenizer,
    PreTrainedModel,
    PreTrainedTokenizer,
)

MODEL_CLASSES = {"gpt2": (GPT2Config, GPT2LMHeadModel, GPT2Tokenizer)}

logger = logging.getLogger(__name__)

In [15]:
import string
corpus_cleaned = ''.join(filter(lambda x: x in string.printable, corpus))

In [16]:
class SportCommentaryData(Dataset):
    def __init__(
        self,
        tokenizer: PreTrainedTokenizer,
        #file_path: str,
        block_size=512,
        overwrite_cache=False,
    ):
        #assert os.path.isfile(file_path)

        block_size = block_size - (
            tokenizer.model_max_length - tokenizer.max_len_single_sentence
        )

        # change if args are added at later point
        cached_features_file = os.path.join(
           "gpt2" + "_" + str(block_size) + "_file.txt" 
        )

        if os.path.exists(cached_features_file) and not overwrite_cache:
            logger.info(
                f"Loading features from your cached file {cached_features_file}"
            )
            with open(cached_features_file, "rb") as cache:
                self.examples = pickle.load(cache)
                logger.debug("Loaded examples from cache")
        else:
            logger.info(f"Creating features from file")

            self.examples = []

            text = corpus
            tokenized_text = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text))

            for i in range(0, len(tokenized_text) - block_size + 1, block_size):
                self.examples.append(
                    tokenizer.build_inputs_with_special_tokens(
                        tokenized_text[i : i + block_size]
                    )
                )

            logger.info(f"Saving features into cached file {cached_features_file}")
            with open(cached_features_file, "wb") as cache:
                
                pickle.dump(self.examples, cache, protocol=pickle.HIGHEST_PROTOCOL)

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, item):
        return torch.tensor(self.examples[item], dtype=torch.long)

In [17]:
device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')
model = model.to(device)

dataset = SportCommentaryData(tokenizer= tokenizer )
commentary_loader = DataLoader(dataset,batch_size=1,shuffle=True)

BATCH_SIZE = 1
EPOCHS = 1
LEARNING_RATE = 0.0002
WARMUP_STEPS = 2000

from transformers import AdamW, get_linear_schedule_with_warmup

model = model.to(device)
model.train()
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=WARMUP_STEPS, num_training_steps=-1)
script_count = 0
sum_loss = 0.0
batch_count = 0

for epoch in range(EPOCHS):
    print(f"EPOCH {epoch} started" + '=' * 30)
    for idx,script in enumerate(commentary_loader):
        outputs = model(script.to(device), labels=script.to(device))
        loss, logits = outputs[:2]                        
        loss.backward()
        sum_loss = sum_loss + loss.detach().data
                       
        script_count = script_count + 1
        if script_count == BATCH_SIZE:
            script_count = 0    
            batch_count += 1
            optimizer.step()
            scheduler.step() 
            optimizer.zero_grad()
            model.zero_grad()
            
        if batch_count == 100:
            model.eval()
            print(f"sum loss {sum_loss}")
            sample_outputs = model.generate(
                                    bos_token_id=random.randint(1,30000),
                                    do_sample=True,   
                                    top_k=50, 
                                    max_length = 1000,
                                    top_p=0.95, 
                                    num_return_sequences=1
                                )

            print("Output:\n" + 100 * '-')
            for i, sample_output in enumerate(sample_outputs):
                  print("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))
            
            batch_count = 0
            sum_loss = 0.0
            model.train()
output_dir = '/content/drive/MyDrive/ANLP/Model/Model_Five'

from transformers import WEIGHTS_NAME, CONFIG_NAME
output_model_file = os.path.join(output_dir, WEIGHTS_NAME)
output_config_file = os.path.join(output_dir, CONFIG_NAME)

torch.save(model.state_dict(), output_model_file)
model.config.to_json_file(output_config_file)
tokenizer.save_vocabulary(output_dir)

ValueError: ignored

In [ ]:
output_dir = '/content/drive/MyDrive/ANLP/Model/Model_Five'
model = GPT2LMHeadModel.from_pretrained(output_dir)
tokenizer = GPT2Tokenizer.from_pretrained(output_dir)

input_ids = tokenizer.encode('Its five runs', return_tensors='pt')

sample_outputs = model.generate(
                        input_ids= input_ids,
                        do_sample = True,
                        #num_beams= 5,
                        max_length = 100,
                        top_k = 50,
                        top_p=0.85, 
                        num_return_sequences=1
                    )

print("Output:\n" + 100 * '-')
for i, sample_output in enumerate(sample_outputs):
      print("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))

In [ ]:
from nltk.translate.bleu_score import sentence_bleu
reference = "Sam Curran to Dhoni, wide, fuller and slipped down leg-side. MS allows the ball to go through to the keeper. Ashwin has reviewed this. Was it a nick down the leg-side? It was wided and Ashwin out of the blue decided to review. Nothing of the UltraEdge and it stays as a wide. That was bizarre"
genererated = "Its a wide ball to deep mid-wicket, Karthik gets down low and pulls it wide of long-off. Just in front of square, and the man running past was the fielder in the deep S Kaul to Ishan Kishan, FOUR, massive, too full and too straight, Kishan goes after it with a lofted pull through the line, the timing was in sync with the delivery and it raced away through the lineIsh Sodhi"
reference_texts = tokenizer.encode(reference, return_tensors='pt')
generated_texts = tokenizer.encode(genererated, return_tensors='pt')

# Compute BLEU score for each generated text
bleu_scores = []
for ref, gen in zip(reference_texts, generated_texts):
    bleu = sentence_bleu([ref], gen)
    bleu_scores.append(bleu)

# Calculate average BLEU score
avg_bleu = sum(bleu_scores) / len(bleu_scores)
print("Average BLEU score:", avg_bleu)